In [105]:
import requests
import sqlite3 as lite
import datetime
from datetime import timedelta

In [106]:
r = requests.get('https://api.forecast.io/forecast/e3e079c19e476c3b60ae53759759c063/40.778996,-111.932630,2016-04-25T21:22:25')

In [107]:
r.status_code

200

In [108]:
r.headers['content-type']

'application/json; charset=utf-8'

In [109]:
r.json().keys()

[u'hourly',
 u'currently',
 u'longitude',
 u'flags',
 u'daily',
 u'offset',
 u'latitude',
 u'timezone']

In [110]:
cities = { "Salt_Lake_City": '40.778996,-111.932630',
            "Austin": '30.303936,-97.754355',
            "New_Orleans": '30.053420,-89.934502',
            "Seattle": '47.620499,-122.350876',
            "Phoenix": '33.572154,-112.090132'
        }

In [111]:
con = lite.connect('weather.db')
cur = con.cursor()

In [112]:
import pandas as pd

In [113]:
start_date = datetime.datetime.now() - datetime.timedelta(days=30)

In [114]:
print(start_date)

2016-05-03 21:54:28.792000


In [115]:
end_date = datetime.datetime.now()

In [116]:
print(end_date)
print(end_date.strftime('%Y%m%d'))

2016-06-02 21:54:28.854000
20160602


In [117]:
pd.date_range(start_date, end_date)

cur.execute('DROP TABLE IF EXISTS daily_temp')
cur.execute('CREATE TABLE daily_temp(day_of_reading INT, Salt_Lake_City REAL, Austin REAL, New_Orleans REAL, Seattle REAL, Phoenix REAL);')

In [118]:
api_source = 'https://api.forecast.io/forecast/'
api_key = 'e3e079c19e476c3b60ae53759759c063/'
days = 30

for i in range(30):
    query_date = end_date - datetime.timedelta(days)
    days -= 1

    with con:
        cur.execute("INSERT into daily_temp(day_of_reading) VALUES (?)", (int(query_date.strftime('%Y%m%d')),))

    for k,v in cities.iteritems():
        r = requests.get(api_source + api_key + v + ',' + query_date.strftime('%Y-%m-%dT%H:%M:%S'))
        with con:
            cur.execute('UPDATE daily_temp SET ' + k + ' = ' + str(r.json()['daily']['data'][0]['temperatureMax']) + ' WHERE day_of_reading = ' + query_date.strftime('%Y%m%d'))
con.close()
    